In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['NEO4J_USERNAME'] = os.getenv('NEO4J_USERNAME')
os.environ['NEO4J_PASSWORD'] = os.getenv('NEO4J_PASSWORD')
os.environ['NEO4J_URI'] = os.getenv('NEO4J_URI')



## If using GROQ Client
GROQ_API_KEY=os.environ['GROQ_API_KEY']
## If using OpenAI Client
#OPENAI_API_KEY="openai_api_key"
## If using Neo4j
NEO4J_USERNAME=os.environ['NEO4J_USERNAME'] 
NEO4J_PASSWORD=os.environ['NEO4J_PASSWORD'] 
NEO4J_URI=os.environ['NEO4J_URI']

In [2]:
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random

## Input data directory
data_dir = "cureus"
inputdirectory = Path(f"./data_input/{data_dir}")
## This is where the output csv files will be written
out_dir = data_dir
outputdirectory = Path(f"./data_output/{out_dir}")

In [3]:
loader = DirectoryLoader(inputdirectory, show_progress=True)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
print(pages[3].page_content)


100%|██████████| 1/1 [00:01<00:00,  1.36s/it]

Number of chunks =  23
An extensive literature search was performed, and 56 articles published in peer-reviewed journals between 2005 and 2021 were selected and analyzed. The corresponding authors' experiential knowledge served as the foundation for the analysis.


In [4]:
from knowledge_graph_maker import GraphMaker, Ontology, GroqClient
from knowledge_graph_maker import Document


ontology = Ontology(
    # labels of the entities to be extracted. Can be a string or an object, like the following.
    labels=[
        {"Person": "Person name without any adjectives, Remember a person may be references by their name or using a pronoun"},
        {"Object": "Do not add the definite article 'the' in the object name"},
        {"Event": "Event event involving multiple people. Do not include qualifiers or verbs like gives, leaves, works etc."},
        "Place",
        "Document",
        "Organisation",
        "Action",
        {"Miscellanous": "Any important concept can not be categorised with any other given label"},
    ],
    # Relationships that are important for your application.
    # These are more like instructions for the LLM to nudge it to focus on specific relationships.
    # There is no guarentee that only these relationships will be extracted, but some models do a good job overall at sticking to these relations.
    relationships=[
        "Relation between any pair of Entities",
        ],
)

In [5]:
## Groq models
model = "mixtral-8x7b-32768"
# model ="llama3-8b-8192"
# model = "llama3-70b-8192"
# model="gemma-7b-it"

## Open AI models
#oai_model="gpt-3.5-turbo"

## Use Groq
llm = GroqClient(model=model, temperature=0.1, top_p=0.5)
## OR Use OpenAI
# llm = OpenAIClient(model=oai_model, temperature=0.1, top_p=0.5)
graph_maker = GraphMaker(ontology=ontology, llm_client=llm, verbose=False)

In [6]:
pages[0]

Document(page_content="Abstract India’s health indicators have improved in recent times but continue to lag behind those of its peer nations. The country with a population of 1.3 billion, has an estimated active health workers density of doctors and nurses/midwives of 5.0 and 6.0 respectively, for 10,000 persons, which is much lower than the WHO threshold of 44.5 doctors, nurses, and midwives per 10,000 population. The issue is compounded by the skewed inter-state, urban-rural, and public-private sector divide. Calls to urgently augment the skilled health workforce reinforce the central role human resources have in healthcare, which has evolved into a complex multifactorial issue. The paucity of skilled personnel must be addressed if India is to accelerate its progress toward achieving universal health coverage and its sustainable development goals (SDGs). The recent increase in the federal health budget offers an unprecedented opportunity to do this. This article utilizes the ready ma

In [7]:
## create a graph out of a list of Documents.
graph = graph_maker.from_documents(
    list(pages),
    delay_s_between=10 ## delay_s_between because otherwise groq api maxes out pretty fast.
    )
## result -> a list of Edges.
print("Total number of Edges", len(graph))


▶︎ GRAPH MAKER LOG - 2024-06-24 11:02:46 - INFO 
Document: 1


▶︎ GRAPH MAKER VERBOSE - 2024-06-24 11:02:46 - INFO 
Using Ontology:
labels=[{'Person': 'Person name without any adjectives, Remember a person may be references by their name or using a pronoun'}, {'Object': "Do not add the definite article 'the' in the object name"}, {'Event': 'Event event involving multiple people. Do not include qualifiers or verbs like gives, leaves, works etc.'}, 'Place', 'Document', 'Organisation', 'Action', {'Miscellanous': 'Any important concept can not be categorised with any other given label'}] relationships=['Relation between any pair of Entities']



Using Model:  mixtral-8x7b-32768


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-06-24 11:02:48 - INFO 
LLM Response:
[
 {
"node_1": {"label": "Person", "name": "authors"},
"node_2": {"label": "Miscellanous", "name": "READ approach"},
"relationship": "The authors use the READ approach to analyze the health system in India."
},
{
"node_1": {"label": "Miscellanous", "name": "READ approach"},
"node_2": {"label": "Action", "name": "analyze"},
"relationship": "The READ approach is used to analyze the health system in India."
},
{
"node_1": {"label": "Miscellanous", "name": "READ approach"},
"node_2": {"label": "Action", "name": "extract and analyze data"},
"relationship": "The READ approach involves extracting and analyzing data."
},
{
"node_1": {"label": "Miscellanous", "name": "READ approach"},
"node_2": {"label": "Action", "name": "distill findings"},
"relationship": "The READ approach involves distilling findings."
},
{
"node_1": {"label": "

Using Model:  mixtral-8x7b-32768


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-06-24 11:03:00 - INFO 
LLM Response:
[
 {
 "node_1": {"label": "Organisation", "name": "Public Health"},
 "node_2": {"label": "Miscellanous", "name": "Health Policy"},
 "relationship": "Public Health is a domain that the Health Policy is related to."
 },
 {
 "node_1": {"label": "Event", "name": "Epidemiology/Public Health"},
 "node_2": {"label": "Miscellanous", "name": "Health Policy"},
 "relationship": "Epidemiology/Public Health is a field of study that the Health Policy may be related to."
 },
 {
 "node_1": {"label": "Organisation", "name": "Health Sector Reform"},
 "node_2": {"label": "Miscellanous", "name": "Health Policy"},
 "relationship": "Health Sector Reform is an area of focus that the Health Policy may involve."
 },
 {
 "node_1": {"label": "Person", "name": "Auxiliary Nurse Midwives"},
 "node_2": {"label": "Organisation", "name": "Health Policy"},
 

...

Using Model:  mixtral-8x7b-32768


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:groq._base_client:Retrying request to /openai/v1/chat/completions in 8.000000 seconds
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-06-24 11:08:19 - INFO 
LLM Response:
[
 {
 "node_1": {"label": "Person", "name": "physicians"},
 "node_2": {"label": "Action", "name": "attending"},
 "relationship": "Physicians are the ones who attend patients as per the context."
 },
 {
 "node_1": {"label": "Person", "name": "physicians"},
 "node_2": {"label": "Miscellanous", "name": "low physician-to-patient ratio"},
 "relationship": "A low physician-to-patient ratio results in delays in attending patients by physicians."
 },
 {
 "node_1": {"label": "Person", "name": "physicians"},
 "node_2": {"label": "Miscellanous", "name": "costs"},
 "relationship": "A low physician-to-patient ratio adds to costs for physic

Using Model:  mixtral-8x7b-32768


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:groq._base_client:Retrying request to /openai/v1/chat/completions in 4.000000 seconds
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-06-24 11:08:34 - INFO 
LLM Response:
[
 {
 "node_1": {"label": "Event", "name": "Conclusions"},
 "node_2": {"label": "Miscellanous", "name": "aims, strategy, structure, and service provision"},
 "relationship": "The 'Conclusions' discuss the 'aims, strategy, structure, and service provision' of the public and private health sector."
 },
 {
 "node_1": {"label": "Event", "name": "Conclusions"},
 "node_2": {"label": "Action", "name": "synergize their services"},
 "relationship": "The 'Conclusions' recommend 'synergizing the services' of the public and private health sector."
 },
 {
 "node_1": {"label": "Event", "name": "Conclusions"},
 "node_2": {"label": "Miscellan

Total number of Edges 280


In [8]:
from knowledge_graph_maker import Neo4jGraphModel

create_indices = True
neo4j_graph = Neo4jGraphModel(edges=graph, create_indices=create_indices)

neo4j_graph.save()


aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
a

280

In [9]:
from helpers.df_helpers import graph2Df
dfg1 = graph2Df(graph)
dfg1.to_csv(outputdirectory/"graph.csv", sep="|", index=False)

ModuleNotFoundError: No module named 'ollama'